<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/LandmarkExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 41.3 MB/s eta 0:00:00


In [ ]:
!unzip "fer2013.zip"

In [3]:
import plotly.express as px 
import cv2
import mediapipe as mp

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy import linspace
from mpl_toolkits import mplot3d
import glob, os
import json
import pandas as pd

In [ ]:

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:

    df = pd.DataFrame(columns=['nome', 'landmarks', 'label'])

    for root, dirs, files in os.walk("train"):
      for dir in dirs:
 
        label=dir
        for _file in glob.glob("train/"+dir+"/*.jpg"):
          
            image = cv2.imread(_file)
        # Convert the BGR image to RGB before processing.
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            

            annotated_image = image.copy()
            if(results.multi_face_landmarks!=None):
              for face_landmarks in results.multi_face_landmarks:
                #print('face_landmarks:', face_landmarks)
                mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
              points_array = np.array(face_landmarks)
              
              x_array=[]
              y_array=[]
              z_array=[]

              landmark_matrix = []
              for data_point in face_landmarks.landmark:
                landmark_matrix.append([data_point.x, data_point.y, data_point.z])

              df = pd.concat([df,pd.DataFrame([[_file, landmark_matrix, label]], columns = df.columns)]) 
              #df.append([_file, landmark_matrix, label])
                    

    
    df.to_csv("training_set.csv", index = False)  
    
          #print(keypoints)
    from google.colab import files

    files.download("training_set.csv")